In [18]:
# Richard Tobing 862219395


# Task List:
    # [x] Misplaced Tiles heuristic: number of misplaced tiles
    # [x] Manhattan heuristic: the number of moves each misplaced tile needs to take to get to the correct placement
    # [x]  MAKE-QUEUE: takes in the initial state, and returnsn the frontier
        # [x] simple movement 
        # [x] corners 
        # [x] sides and center
    # [] Misplaced queueing function
    # [] Mahtattan queueing function
    # [] UCS queueing function
    # [] Problem obj: has OPERATORS, GOAL_TEST(), and INITIAL_STATE as fields
    # [] REMOVE_FRONT(nodes): chooses one node from the frontier to process
    # [] Uniform Cost Search: essentially equivalent to BFS since all moves only cost 1, basically a move counter
        # [] skip repeated states
    # [] user input


    #[x] optimize makeQueue
    #[] add good comments
    #[] find brown m&ms



# -----these are sample problems taken from Dr. Keogh's slides:-----
# 3 misplaced, 8 manhattan:
sample=    [[3,2,8],
            [4,5,6],
            [7,1,0]]
mySample=  [[6,6,6],
            [6,6,6],
            [6,6,0]]
sampleD4=  [[1,2,3],
            [5,0,6],
            [4,7,8]]
sample2=   [[1,2,3],
            [4,5,8],
            [6,7,0]]
sampleD2=  [[1,2,3],
            [4,5,6],
            [0,7,8]]
sampleD0=  [[1,2,3],
            [4,5,6],
            [7,8,0]]
sampleD8=  [[1,3,6],
            [5,0,2],
            [4,7,8]]
sampleD12=  [[1,3,6],
            [5,0,7],
            [4,8,2]]
#-----------------------------------------------------------------///
            
#----problem proto-Object-------
initState= sampleD2
dim= len(initState[0])
squize= dim*dim
stateBuffer= []
#----------------------------


def heuristic(matrix):
    misplaced, lindex, manhattan= 0,0,0
    isGoal= False
    zeroPos= (-1,-1)
    for y in range(dim):
        for x in range(dim):
            i = matrix[y][x]
            pos= (y,x)
            lindex+= 1
            if lindex != i:
                misplaced+= 1
                dif= abs(i - lindex) if i!=0 else abs(squize - lindex)
                manhattan+= (dif//dim) + (dif%dim)
            if i == 0:
                zeroPos= pos
    if matrix[dim-1][dim-1]==0:
        misplaced-= 1
    if misplaced == 0:
        isGoal= True
    return misplaced, manhattan, zeroPos, isGoal



def shiftSquare(node,y,x,yShift,xShift):
    swap= node[y+yShift][x+xShift]
    node[y+yShift][x+xShift]= 0
    node[y][x]= swap
    return node

def makeQueue(matrix, zeroPos):
    frontier= []
    y= zeroPos[0]
    x=zeroPos[1]
    
    node1 = [row[:] for row in matrix]
    node2 = [row[:] for row in matrix]
    if y==0:
        #[x]--top left--
        if x==0:
            # shift down
            frontier.append(shiftSquare(node1,y,x,1,0))
            # shift right
            frontier.append(shiftSquare(node2,y,x,0,1))

        #[x] --top right--
        elif x==(dim-1):
            #shift down
            frontier.append(shiftSquare(node1,y,x,1,0))
            #shift left
            frontier.append(shiftSquare(node2,y,x,0,-1))
        
        # [x] --top side--
        else:
            node3= [row[:] for row in matrix]
            #shift down
            frontier.append(shiftSquare(node1,y,x,1,0))
            # shift right
            frontier.append(shiftSquare(node2,y,x,0,1))
            #shift left
            frontier.append(shiftSquare(node3,y,x,0,-1))

    elif y== (dim-1):
        #[x]--bottom right-- 
        if x== (dim-1):
            #shift left
            frontier.append(shiftSquare(node1,y,x,0,-1))
            #shift up
            frontier.append(shiftSquare(node2,y,x,-1,0))

        #[x]--botom left--
        elif x== 0:
            # shift up
            frontier.append(shiftSquare(node1,y,x,-1,0))
            # shift right
            frontier.append(shiftSquare(node2,y,x,0,1))
        
        # [x] --bottom side--
        else:
            node3= [row[:] for row in matrix]
            # shift up
            frontier.append(shiftSquare(node1,y,x,-1,0))
            # shift right
            frontier.append(shiftSquare(node2,y,x,0,1))
            #shift left
            frontier.append(shiftSquare(node3,y,x,0,-1))
    else:
        #[]--left side--
        node3= [row[:] for row in matrix]
        if x==0:
            # shift up
            frontier.append(shiftSquare(node1,y,x,-1,0))
            # shift right
            frontier.append(shiftSquare(node2,y,x,0,1))
            #shift down
            frontier.append(shiftSquare(node3,y,x,1,0))
            
        #[] --right side--
        elif x == (dim-1):
            # shift up
            frontier.append(shiftSquare(node1,y,x,-1,0))
            #shift left
            frontier.append(shiftSquare(node2,y,x,0,-1))
            #shift down
            frontier.append(shiftSquare(node3,y,x,1,0))

        # --center region--
        else:
            node4= [row[:] for row in matrix]
            # shift up
            frontier.append(shiftSquare(node1,y,x,-1,0))
            #shift left
            frontier.append(shiftSquare(node2,y,x,0,-1))
            #shift down
            frontier.append(shiftSquare(node3,y,x,1,0))
            # shift right
            frontier.append(shiftSquare(node4,y,x,0,1))
    return frontier
        

def uniformCostSearch(node, nodes, idx, d,zeroPos, grandParents, length):
    if idx==0: 
        newNodes= []
        d+=1
        if (d%2)>=1: grandParents= nodes
    newFrontier= makeQueue(node, zeroPos)
    # newNodes+= [row[:] for row in newFrontier]
    for m in newFrontier:
        if m not in grandParents:
            newNodes.append(m)
    if idx== (length-1):
        #print("g: " + str(g) + "  L: " + str(length) + " idx: " + str(idx))
        #print(nodes)
        nodes = newNodes
        #print('----')
        #print(nodes)
    return nodes


def uniformCostSearch(node, newNodes, zeroPos):  
    newFrontier= makeQueue(node, zeroPos)
    newNodes+= [row[:] for row in newFrontier]
    return newNodes

def printNode(matrix):
    print(matrix[0])
    print(matrix[1])
    print(matrix[2])
    #print(h[0], h[1],h[3])
    print("---------\n")


#--------driver proto-function---------
def generalSearch(matrix):
    finalState= False
    g,d,queueingFuncion= 0,0,0
    minHattan, minPlaced= squize, squize 
    nodes= []
    nodes.append(matrix)

    #nodes= makeQueue(matrix, h[2])
    while finalState==False:
        length= len(nodes)
        if length==0: 
            return "failure"

        newNodes= []
        for node in nodes:
            h= heuristic(node)
            misplaced= h[0]
            manhattan= h[1]
            zeroPos= h[2]
            isGoal= h[3]

            if isGoal== True: 
                finalState = True
                printNode(node)
                print("g: " + str(g) + "  L: " + str(length) + "  Final Node: " + str(node) + "  d: " + str(length))
                return nodes, g, length

            idx= nodes.index(node)
            # if misplaced<minPlaced:
            #     minPlaced= misplaced
            #     mindex= idx
            # if manhattan<minHattan:
            #     minHattan= manhattan
            #     mandex= idx
            

            # Djiskstra/Uniform Cost Search will behave like BFS because the cost is alwasy the same, so it will expand every node in the frontier
            uniformCostSearch(node, newNodes, zeroPos)

    
  
            # A* search with misplaced heuristic will only expand the node with the least misplaced tiles

            # A* Seach with manhattan heuristic will only expand the node with the nearest manhattan distance

            # Because the cost of every move is 1, we only need to increment g by 1 for each expansion
            g+=1

        # after descending each level of the tree, we refresh the nodes in our queue to avoid time and space issues, each level will have a different queue
        nodes = []
        [nodes.append(x) for x in newNodes if x not in nodes]
        d+=1

    return "failure"
        


#---------------------------------------



#print(generalSearch(sampleD0))
#print(generalSearch(sampleD2))
generalSearch(sampleD12)
#print(generalSearch(sampleD8))
 



[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
---------

g: 3754  L: 1561  Final Node: [[1, 2, 3], [4, 5, 6], [7, 8, 0]]  d: 1561


([[[1, 3, 6], [5, 0, 7], [4, 8, 2]],
  [[1, 6, 0], [5, 3, 7], [4, 8, 2]],
  [[0, 1, 6], [5, 3, 7], [4, 8, 2]],
  [[0, 3, 6], [1, 5, 7], [4, 8, 2]],
  [[1, 3, 6], [4, 5, 7], [0, 8, 2]],
  [[1, 3, 6], [5, 8, 7], [4, 2, 0]],
  [[1, 3, 6], [5, 8, 7], [0, 4, 2]],
  [[1, 3, 0], [5, 7, 6], [4, 8, 2]],
  [[1, 3, 6], [5, 7, 2], [4, 8, 0]],
  [[1, 6, 7], [5, 0, 3], [4, 8, 2]],
  [[1, 6, 7], [5, 3, 2], [4, 8, 0]],
  [[5, 1, 6], [3, 0, 7], [4, 8, 2]],
  [[5, 1, 6], [4, 3, 7], [0, 8, 2]],
  [[3, 5, 6], [1, 0, 7], [4, 8, 2]],
  [[3, 6, 0], [1, 5, 7], [4, 8, 2]],
  [[1, 3, 6], [4, 0, 7], [8, 5, 2]],
  [[1, 3, 6], [4, 5, 7], [8, 2, 0]],
  [[1, 3, 0], [5, 8, 6], [4, 2, 7]],
  [[1, 3, 6], [5, 0, 8], [4, 2, 7]],
  [[0, 3, 6], [1, 8, 7], [5, 4, 2]],
  [[1, 3, 6], [8, 0, 7], [5, 4, 2]],
  [[1, 7, 3], [5, 0, 6], [4, 8, 2]],
  [[0, 1, 3], [5, 7, 6], [4, 8, 2]],
  [[1, 3, 6], [5, 0, 2], [4, 7, 8]],
  [[1, 3, 6], [5, 7, 2], [0, 4, 8]],
  [[1, 7, 0], [5, 6, 3], [4, 8, 2]],
  [[0, 1, 7], [5, 6, 3], [4, 8, 2]],
 